### Pinecone - Symentic search with metadata filtering
- News articles
- symbol, publishedDate,title,site

In [ ]:
!pip install pinecone-client sentence-transformers

In [7]:
import time
import pandas as pd
import numpy as np

import pinecone
from sentence_transformers import SentenceTransformer

In [8]:
pinecone.init(
    api_key="294f995e-d4b5-4447-8cdf-788f4d723758",
    environment="gcp-starter"
)

### Prepare Data - News Articles

In [42]:
df1 = pd.read_csv('news.txt')
df1

,symbol,publishedDate,title,site
0,WMT,2023-11-16 16:51:07,Walmart aims to lower food costs to entice cus...,Fox Business
1,CACC,2023-11-16 16:50:10,Credit Acceptance: There Are Better Opportunities,Seeking Alpha
2,GPS,2023-11-16 16:49:00,Gap shares rally as signs of discounting ease,Market Watch
3,ARM,2023-11-16 16:48:20,Arm Holdings Has This Massive Growth Opportuni...,The Motley Fool
4,AMAT,2023-11-16 16:47:52,Applied Materials stock slumps despite 4Q beat...,Proactive Investors
...,...,...,...,...
221,EG,2023-11-16 12:47:51,Pick These 4 Stocks With Excellent Interest Co...,Zacks Investment Research
222,ENPH,2023-11-16 12:47:50,Enphase (ENPH) Starts Shipping IQ8P Microinver...,Zacks Investment Research
223,SR,2023-11-16 12:47:49,"Spire (SR) Q4 Loss Wider Than Expected, Sales ...",Zacks Investment Research
224,ES,2023-11-16 12:47:47,Eversource (ES) to Gain From Investments & Ren...,Zacks Investment Research


In [44]:
df1['title'] = df1['title'].fillna('')

In [45]:
df1

,symbol,publishedDate,title,site
0,WMT,2023-11-16 16:51:07,Walmart aims to lower food costs to entice cus...,Fox Business
1,CACC,2023-11-16 16:50:10,Credit Acceptance: There Are Better Opportunities,Seeking Alpha
2,GPS,2023-11-16 16:49:00,Gap shares rally as signs of discounting ease,Market Watch
3,ARM,2023-11-16 16:48:20,Arm Holdings Has This Massive Growth Opportuni...,The Motley Fool
4,AMAT,2023-11-16 16:47:52,Applied Materials stock slumps despite 4Q beat...,Proactive Investors
...,...,...,...,...
221,EG,2023-11-16 12:47:51,Pick These 4 Stocks With Excellent Interest Co...,Zacks Investment Research
222,ENPH,2023-11-16 12:47:50,Enphase (ENPH) Starts Shipping IQ8P Microinver...,Zacks Investment Research
223,SR,2023-11-16 12:47:49,"Spire (SR) Q4 Loss Wider Than Expected, Sales ...",Zacks Investment Research
224,ES,2023-11-16 12:47:47,Eversource (ES) to Gain From Investments & Ren...,Zacks Investment Research


### Create Embeddings and Vectors

In [9]:
# average_word_embeddings_komninos
# dimension = 300

In [43]:
model = SentenceTransformer('average_word_embeddings_komninos')

(…)899125e041c5f1b0c1f371716/.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/267M [00:00<?, ?B/s]

(…)beddings/whitespacetokenizer_config.json:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

(…)WordEmbeddings/wordembedding_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

(…)041c5f1b0c1f371716/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)1f8dd899125e041c5f1b0c1f371716/README.md:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

(…)371716/config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

(…)dd899125e041c5f1b0c1f371716/modules.json:   0%|          | 0.00/248 [00:00<?, ?B/s]

In [46]:
model

SentenceTransformer(
  (0): WordEmbeddings(
    (emb_layer): Embedding(222305, 300)
  )
  (1): Pooling({'word_embedding_dimension': 300, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [47]:
print('Ecoding titles...')
encode_titles = model.encode(df1['title'].tolist(), show_progress_bar=True)

Ecoding titles...


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [48]:
df1['title_encoded'] = encode_titles.tolist()

In [49]:
df1

,symbol,publishedDate,title,site,title_encoded
0,WMT,2023-11-16 16:51:07,Walmart aims to lower food costs to entice cus...,Fox Business,"[0.16963566839694977, 0.1335783451795578, -0.1..."
1,CACC,2023-11-16 16:50:10,Credit Acceptance: There Are Better Opportunities,Seeking Alpha,"[0.05878175050020218, -0.011991247534751892, -..."
2,GPS,2023-11-16 16:49:00,Gap shares rally as signs of discounting ease,Market Watch,"[0.11598699539899826, -0.0673118308186531, -0...."
3,ARM,2023-11-16 16:48:20,Arm Holdings Has This Massive Growth Opportuni...,The Motley Fool,"[0.2478489875793457, 0.1705268770456314, -0.03..."
4,AMAT,2023-11-16 16:47:52,Applied Materials stock slumps despite 4Q beat...,Proactive Investors,"[0.1731574982404709, 0.06479299813508987, -0.1..."
...,...,...,...,...,...
221,EG,2023-11-16 12:47:51,Pick These 4 Stocks With Excellent Interest Co...,Zacks Investment Research,"[0.1376696676015854, -0.06740499287843704, -0...."
222,ENPH,2023-11-16 12:47:50,Enphase (ENPH) Starts Shipping IQ8P Microinver...,Zacks Investment Research,"[0.030173001810908318, 0.03865500167012215, 0...."
223,SR,2023-11-16 12:47:49,"Spire (SR) Q4 Loss Wider Than Expected, Sales ...",Zacks Investment Research,"[0.25653210282325745, -0.031999003142118454, -..."
224,ES,2023-11-16 12:47:47,Eversource (ES) to Gain From Investments & Ren...,Zacks Investment Research,"[0.29149940609931946, -0.021570801734924316, -..."


In [ ]:
# check dimension

### Setup Pinecone
- Make sure the dimension is same as per model used

In [50]:
pinecone.list_indexes()

['test']

In [51]:
pinecone.delete_index('test')

In [53]:
pinecone.create_index('newstitles', dimension=300, metric='cosine')

In [54]:
pinecone.list_indexes()

['newstitles']

In [55]:
index = pinecone.Index('newstitles')

### Upsert vector data

In [ ]:
# index.upsert(vectors=(id, vector))

In [61]:
for i in range(0,20):
  ids = str(df1.index[i])
  vecs = df1.title_encoded.iloc[i],
  metadata = {
      'symbol': df1.symbol.iloc[i],
      'publishedDate': df1.publishedDate.iloc[i],
      'title': df1.title.iloc[i],
      'site': df1.site.iloc[i]
  }
  upsert_response = index.upsert(vectors=[(ids, vecs, metadata)])

In [57]:
ids

'19'

In [59]:
len(vecs)

1

In [60]:
metadata

{'symbol': 'SCWX',
 'publishedDate': '2023-11-16 16:35:00',
 'title': 'Secureworks to Report Third Quarter Fiscal 2024 Financial Results on December 7, 2023',
 'site': 'PRNewsWire'}

In [62]:
index.describe_index_stats()

{'dimension': 300,
 'index_fullness': 0.0002,
 'namespaces': {'': {'vector_count': 20}},
 'total_vector_count': 20}

### Query

In [ ]:
phrase = "food cost"
encode_phrase = model.encode(phrase).tolist()
encode_phrase

In [66]:
top_k = 20
result = index.query(
    vector= encode_phrase,
    top_k = top_k,
    include_metadata = True
)
result

{'matches': [{'id': '0',
              'metadata': {'publishedDate': datetime.datetime(2023, 11, 16, 16, 51, 7),
                           'site': 'Fox Business',
                           'symbol': 'WMT',
                           'title': 'Walmart aims to lower food costs to '
                                    'entice customers to buy other products'},
              'score': 0.727331102,
              'values': []},
             {'id': '18',
              'metadata': {'publishedDate': datetime.datetime(2023, 11, 16, 16, 35, 42),
                           'site': 'Investopedia',
                           'symbol': 'M',
                           'title': "Macy's Shares Jump After Earnings Beat as "
                                    'It Raised Margins, Cut Costs, and Shed '
                                    'Inventory'},
              'score': 0.570147336,
              'values': []},
             {'id': '17',
              'metadata': {'publishedDate': datetime.datetime(202

In [71]:
result_ids = [int(result.matches[i].id) for i in range(top_k)]

In [73]:
result_ids

[0, 18, 17, 7]

In [74]:
df1.loc[result_ids]

,symbol,publishedDate,title,site,title_encoded
0,WMT,2023-11-16 16:51:07,Walmart aims to lower food costs to entice cus...,Fox Business,"[0.16963566839694977, 0.1335783451795578, -0.1..."
18,M,2023-11-16 16:35:42,Macy's Shares Jump After Earnings Beat as It R...,Investopedia,"[0.20590749382972717, 0.000827297568321228, -0..."
17,ROST,2023-11-16 16:36:00,"Ross sales, profit rise as retailer rides ‘res...",Market Watch,"[0.2094912827014923, -0.0034867185167968273, -..."
7,PYPL,2023-11-16 16:45:10,Best Stocks to Buy: PayPal Stock vs. SoFi Stock,The Motley Fool,"[0.0852697491645813, 0.09646448493003845, -0.0..."


### Query with metadata filters

In [78]:
top_k = 20
result = index.query(
    vector= encode_phrase,
    top_k = top_k,
    include_metadata = True,
    filter = {
        "site": {"$eq": "Market Watch"}
    }
)
result

{'matches': [{'id': '17',
              'metadata': {'publishedDate': datetime.datetime(2023, 11, 16, 16, 36),
                           'site': 'Market Watch',
                           'symbol': 'ROST',
                           'title': 'Ross sales, profit rise as retailer rides '
                                    "‘resilience of the off-price sector'"},
              'score': 0.538372219,
              'values': []},
             {'id': '2',
              'metadata': {'publishedDate': datetime.datetime(2023, 11, 16, 16, 49),
                           'site': 'Market Watch',
                           'symbol': 'GPS',
                           'title': 'Gap shares rally as signs of discounting '
                                    'ease'},
              'score': 0.456594914,
              'values': []}],
 'namespace': ''}

### Cleanup

In [79]:
pinecone.delete_index('newstitles')

In [80]:
pinecone.list_indexes()

[]